#Data Loading and preprocessing

##Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Unzipping

In [ ]:
# !unzip '/content/drive/MyDrive/SMAI Proj/Copy of glove.twitter.27B.zip' -d '/content/drive/MyDrive/SMAI Proj/'

##Loading the glove data

In [ ]:
import os
import numpy as np

root_path = '/content/drive/MyDrive/Colab Notebooks/'

wordVecLength = 50

embeddings_index = {} #initialize dictionary
f = open(os.path.join(root_path,'glove.twitter.27B.'+str(wordVecLength)+'d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float64')
    embeddings_index[word] = coefs
f.close()

## Loading the json data

In [ ]:
import json
# f = open('/content/drive/MyDrive/Colab Notebooks/Project/train_data.json')
f = open('/content/drive/MyDrive/Colab Notebooks/train_data_multilable.json')
train_data = json.load(f)
g= open('/content/drive/MyDrive/Colab Notebooks/test_data_multilable.json')
test_data = json.load(g)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from string import punctuation


english_articles_train = []
english_articles_test = []


for d in train_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_train.append(d)

for d in test_data:
  if d['meta_lang_1'] == 'en' and d['meta_lang_2'] == 'en':
    english_articles_test.append(d)


total_train_articles_odd = []
total_train_articles_even = []
total_test_articles_odd = []
total_test_articles_even = []

y_train = []
y_test = []

for eng in english_articles_train:
  total_train_articles_odd.append(eng['text_1'])
  total_train_articles_even.append(eng['text_2'])
  # y_train.append(eng['score'])
  y = []
  y.append(eng['score'])
  y.append(eng['score_1'])
  y.append(eng['score_2'])
  y.append(eng['score_3'])
  y.append(eng['score_4'])
  y.append(eng['score_5'])
  y.append(eng['score_6'])
  y_train.append(np.array(y))

for eng in english_articles_test:
  total_test_articles_odd.append(eng['text_1'])
  total_test_articles_even.append(eng['text_2'])
  y = []
  y.append(eng['score'])
  y.append(eng['score_1'])
  y.append(eng['score_2'])
  y.append(eng['score_3'])
  y.append(eng['score_4'])
  y.append(eng['score_5'])
  y.append(eng['score_6'])
  y_test.append(np.array(y))




total_articles= total_train_articles_odd+total_train_articles_even+total_test_articles_odd+total_test_articles_even


y_train = np.array(y_train)

#########################################################





'''

tf = TfidfVectorizer(min_df =2)

x_train = tf.fit(total_train_articles)
x_train = tf.transform(total_train_articles)
x_test = tf.transform(total_test_articles)


print(x_train.shape)

print(type(x_train))



X_text_1_train = []
X_text_2_train = []
X_text_1_test = []
X_text_2_test = []


print(x_test.shape)




for j in range(1287):
  X_text_1_train.append(x_train[2*j])
  X_text_2_train.append(x_train[2*j+1])

for j in range(236):
  X_text_1_test.append(x_test[2*j])
  X_text_2_test.append(x_test[2*j+1])

'''

'\n\ntf = TfidfVectorizer(min_df =2)\n\nx_train = tf.fit(total_train_articles)\nx_train = tf.transform(total_train_articles)\nx_test = tf.transform(total_test_articles)\n\n\nprint(x_train.shape)\n\nprint(type(x_train))\n\n\n\nX_text_1_train = []\nX_text_2_train = []\nX_text_1_test = []\nX_text_2_test = []\n\n\nprint(x_test.shape)\n\n\n\n\nfor j in range(1287):\n  X_text_1_train.append(x_train[2*j])\n  X_text_2_train.append(x_train[2*j+1])\n\nfor j in range(236):\n  X_text_1_test.append(x_test[2*j])\n  X_text_2_test.append(x_test[2*j+1])\n\n'

##Tokenizing the articles

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

t = Tokenizer()

t.fit_on_texts(total_articles)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(total_articles)

max_length = 500
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#y = np.array(input['label'])
#print(X.shape, y.shape)
print(X.shape)


encoded_docs_train_odd = t.texts_to_sequences(total_train_articles_odd)
encoded_docs_train_even = t.texts_to_sequences(total_train_articles_even)
encoded_docs_test_odd = t.texts_to_sequences(total_test_articles_odd)
encoded_docs_test_even = t.texts_to_sequences(total_test_articles_even)
X1 = pad_sequences(encoded_docs_train_odd, maxlen=max_length, padding='post')
X2 = pad_sequences(encoded_docs_train_even, maxlen=max_length, padding='post')
X3 = pad_sequences(encoded_docs_test_odd, maxlen=max_length, padding='post')
X4 = pad_sequences(encoded_docs_test_even, maxlen=max_length, padding='post')


(3044, 500)


##Making Embedding Matrix

In [ ]:
embedding_matrix = np.zeros((vocab_size, wordVecLength))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
    

embedding_matrix.shape

(70678, 50)

#Model 1

In [ ]:
from tensorflow.keras.layers import Embedding, Input, Flatten, LSTM, concatenate, Dense, Bidirectional
from tensorflow.keras.models import Model
import numpy as np


In [ ]:
def model1():
    input_1 = Input(shape=(max_length, ))
    input_2 = Input(shape=(max_length, ))
    e = Embedding(vocab_size, wordVecLength, weights=[embedding_matrix], input_length=max_length, trainable=False)

    embedded_input_1 = e(input_1)
    embedded_input_1 = e(input_2)


    #model.add(Bidirectional(LSTM(50, dropout=0.2, return_sequences=True)))
    lstm_1 = LSTM(100, )(embedded_input_1)
    lstm_2 = LSTM(100, )(embedded_input_1)
    merge = concatenate([lstm_1, lstm_2])
    dense = Dense(7)(merge)
    
    model = Model([input_1, input_2], dense)
    model.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
    print(model.summary())

    return(model)

In [ ]:
model = model1()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 500, 50)      3533900     ['input_6[0][0]']                
                                                                                                  
 lstm_4 (LSTM)                  (None, 100)          60400       ['embedding_2[1][0]']            
                                                                                                  
 lstm_5 (LSTM)                  (None, 100)          60400       ['embedding_2[1][0]']            
                                                                                            

In [ ]:

history = model.fit([X1,X2], y_train, epochs=1, batch_size=32)

41/41 [==============================] - 4s 40ms/step - loss: 2.6003 - accuracy: 0.2416


In [ ]:
y_pred = model.predict([X3,X4])

8/8 [==============================] - 1s 15ms/step


In [ ]:
print(y_pred)

[[-0.57890654 -0.3886272  -0.5465421  ... -0.592137   -0.5236194
  -0.4239843 ]
 [-0.8323777  -0.12105134 -0.56728756 ... -0.5884733  -0.68715805
  -0.6269337 ]
 [ 0.98412883  0.8618894   0.39081588 ...  0.3718465   0.27739692
   0.15914942]
 ...
 [ 3.039087    2.2603388   3.2026281  ...  3.1223345   2.1787167
   2.1021874 ]
 [ 3.039087    2.2603388   3.2026281  ...  3.1223345   2.1787167
   2.1021874 ]
 [ 3.039087    2.2603388   3.2026281  ...  3.1223345   2.1787167
   2.1021874 ]]


In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.feature_selection import r_regression
print('The mean absolute error for the model is ', mean_absolute_error(y_test[0],y_pred[0]))
print('The mean squared error for the model is ', mean_squared_error(y_test[0],y_pred[0]))

The mean absolute error for the model is  1.8949375876358576
The mean squared error for the model is  4.232087258236433


#Model 2 Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Input, Flatten, LSTM, concatenate, Dense, Bidirectional
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
def model2():
    input_1 = Input(shape=(max_length, ))
    input_2 = Input(shape=(max_length, ))
    e = Embedding(vocab_size, wordVecLength, weights=[embedding_matrix], input_length=max_length, trainable=False)

    embedded_input_1 = e(input_1)
    embedded_input_1 = e(input_2)


    #model.add(Bidirectional(LSTM(50, dropout=0.2, return_sequences=True)))
    lstm_1 = Bidirectional(LSTM(25, ))(embedded_input_1)
    lstm_2 = Bidirectional(LSTM(25, ))(embedded_input_1)
    merge = concatenate([lstm_1+lstm_2, lstm_1-lstm_2])
    dense = Dense(25)(merge)
    # dense2 = Dense(1)(dense)
    dense2 = Dense(7)(dense)
    
    
    model = Model([input_1, input_2], dense2)
    model.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
    print(model.summary())

    return(model)

In [ ]:
model_ = model2()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 500, 50)      3533900     ['input_8[0][0]']                
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 50)          15200       ['embedding_3[1][0]']            
 )                                                                                                
                                                                                                  
 bidirectional_3 (Bidirectional  (None, 50)          15200       ['embedding_3[1][0]']      

In [ ]:
history2 = model_.fit([X1,X2], y_train, epochs=30, batch_size=8, validation_split=0.1)

Epoch 1/30
145/145 [==============================] - 15s 62ms/step - loss: 1.6047 - accuracy: 0.2617 - val_loss: 1.6283 - val_accuracy: 0.1860
Epoch 2/30
145/145 [==============================] - 7s 51ms/step - loss: 0.9601 - accuracy: 0.2876 - val_loss: 1.6882 - val_accuracy: 0.1550
Epoch 3/30
145/145 [==============================] - 7s 51ms/step - loss: 0.8946 - accuracy: 0.2936 - val_loss: 1.5191 - val_accuracy: 0.4729
Epoch 4/30
145/145 [==============================] - 7s 50ms/step - loss: 0.8596 - accuracy: 0.3066 - val_loss: 1.5478 - val_accuracy: 0.2171
Epoch 5/30
145/145 [==============================] - 7s 51ms/step - loss: 0.8210 - accuracy: 0.3152 - val_loss: 1.5440 - val_accuracy: 0.3876
Epoch 6/30
145/145 [==============================] - 8s 53ms/step - loss: 0.7798 - accuracy: 0.3161 - val_loss: 1.5673 - val_accuracy: 0.3876
Epoch 7/30
145/145 [==============================] - 8s 57ms/step - loss: 0.7456 - accuracy: 0.3247 - val_loss: 1.5339 - val_accuracy: 0.426

In [ ]:
y_pred2 = model_.predict([X3,X4])

8/8 [==============================] - 1s 27ms/step


In [ ]:
print(y_pred2.shape)

(235, 7)


In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.feature_selection import r_regression
print('The mean absolute error for the model is ', mean_absolute_error(y_test[0],y_pred2[0]))
print('The mean squared error for the model is ', mean_squared_error(y_test[0],y_pred2[0]))

The mean absolute error for the model is  0.9721598050424031
The mean squared error for the model is  1.3553577767899436


In [ ]:
from scipy.stats import pearsonr

#print(pearsonr(y_test[0],y_pred[0]))
print(pearsonr(y_test[0],y_pred2[0]))

(0.4626405739824177, 0.2958773352695492)


In [ ]:
y_pred_train = model_.predict([X1,X2])
print(pearsonr(y_pred_train[0],y_pred2[0]))

41/41 [==============================] - 1s 27ms/step
(0.20178374018125378, 0.6643724075823727)
